In [ ]:
from src import PickleManager
from src import Resolution
from src import Camera
from src import Vertex, Vector
from src import Scene
from src.render.post_process.post_process_config import PostProcessConfig
from src.render.loops.progress import PreviewConfig, ProgressDisplay
from src.render.render_config import RenderConfig
from src.render import LinearRayCaster

pm = PickleManager(directory="./pickle_data", verbose=True)

# Camera and scene setup

we can choose resolution of the camera from the predefined resolutions in the Resolution enum or make a custom one

In [ ]:
resolution = Resolution.R144p
#resolution = Resolution.custom(1280, 640)

camera = Camera(
    fov = 60,
    aspect_ratio = resolution.aspect_ratio,
    origin = Vertex(0, 0, 0),
    direction = Vector(0, 0, -1),
    up_hint = Vector(0, 1, 0)
)

print(f"Camera: {camera}")

## setup scene with world and lights

In [ ]:
world = pm.load("world.pkl")
light_lib = pm.load_light_library()

lights = light_lib.get_all_lights()

sphere_scene = Scene(
    camera = camera,
    world = world,
    lights = lights,
    skybox_path = "./skybox/shanghai_4k.hdr"
)

#todo how to handle light lists? should scene have a light library and add light types?
pm.save_scene(sphere_scene, scene_name="spheres")

### Render a preview with no antialiasing, no recursion and no skybox at 144p resolution for speed just to see all objects in the scene

In [ ]:
sphere_scene.validate()
pm.save_scene(sphere_scene, scene_name="spheres")

### Move camera, change FOV and render another preview

In [ ]:
render_configuration = RenderConfig(
    resolution=Resolution.R144p,
    samples_per_pixel=1,
    max_depth=1,
)

preview_configuration = PreviewConfig(
    progress_display=ProgressDisplay.TQDM_IMAGE_PREVIEW,
)

post_process_configuration = PostProcessConfig(
    enabled=False,
    scale_factor=1
)

my_ray_tracer = LinearRayCaster(scene=sphere_scene, render_config=render_configuration, preview_config=preview_configuration, post_process_config=post_process_configuration)

my_ray_tracer.render("images/my_spheres.png")


sphere_scene.move_camera_to(Vertex(0, 1, 3))
sphere_scene.look_at(Vertex(0, 0.7, -3))
sphere_scene.set_camera_fov(60)

my_ray_tracer.render("images/my_spheres.png")

### Move light and render another preview to see effect of light position change

In [ ]:
sphere_scene.translate_light(lights[0], Vector(1, 1, -3))

my_ray_tracer.render("images/my_spheres.png")

# Save the scene for later use

when we are done setting up the scene we can save it for later use

In [ ]:
pm.save_scene(sphere_scene, scene_name="spheres")